# test1.ipynb
First test of the delegational learning rule in the context of the spinal cord project.

Let $S_F, S_P, S_{P-F}, M_C$ denote the regions of the cortical model, and $s_F, s_P, s_{P-F}, m_C$ denote their current activities.
Assume that each unit $u_j$ in $M_C$ is associated with a vector $\bar{v}_j$, and that the ouput of the plant is $p = \sum_j u_j \bar{v}_j$. We also set $s_F = p$. The rule should be such that $p \approx s_P$.

The plant $P$ can be implemented as a population of linear units.
If the connection matrix from $M_C$ to $P$ is called $W$, then
$\bar{v}_j = (W_{1,j}, W_{2,j}, ..., W_{n,j})^T \equiv W_{:,j}$. 

In [1]:
from draculab import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create the neuronal populations
N = 8 # size of each population
net_params = {'min_delay' : 0.01,
              'min_buff_size' : 10 }
SF_params = {'type' : unit_types.sigmoidal,
             'thresh' : 0.,
             'slope' : 1.,
             'init_val' : 0.5,
             'tau' : 0.05 }
SP_params = {'type' : unit_types.source,
             'init_val' : 0.5,
             'function' : lambda x: None }
SPF_params = {'type' : unit_types.sigmoidal,
             'thresh' : 0.,
             'slope' : 1.,
              'init_val' : 0.5,
             'tau' : 0.05 }
P_params = {'type' : unit_types.linear,
            'init_val' : 0.5,
            'tau' : 0.05 }

net = network(net_params)
SF = net.create(N, SF_params)
SP = net.create(N, SP_params)
SPF = net.create(2*N, SPF_params)
P = net.create(N, P_params)

# Create the connections. See note on "detecting layer distances" in cortex wiki
SF--SPF_conn = {'rule' : "one_to_one",
                'delay' : 0.02 }
SF--SPF_syn1 = {'type' : synapse_types.static,
               'init_w' : -1. }
SF--SPF_syn2 = {'type' : synapse_types.static,
               'init_w' : 1. }
SP--SPF_conn = {'rule' : "one_to_one",
                'delay' : 0.02 }
SP--SPF_syn1 = {'type' : synapse_types.static,
               'init_w' : 1. }
SP--SPF_syn2 = {'type' : synapse_types.static,
               'init_w' : -1. }


In [18]:
help(network.connect)

Help on function connect in module network:

connect(self, from_list, to_list, conn_spec, syn_spec)
    Connect units using delayed transmission lines and synaptic connections.
    
    connect the units in the 'from_list' to the units in the 'to_list' using the
    connection specifications in the 'conn_spec' dictionary, and the
    synapse specfications in the 'syn_spec' dictionary.
    
    Args:
        from_list: A list with the IDs of the units sending the connections
    
        to_list: A list the IDs of the units receiving the connections
        
        conn_spec: A dictionary specifying a connection rule, and delays.
            REQUIRED PARAMETERS
            'rule' : a string specifying a rule on how to create the connections. 
                    Currently implemented: 
                    'fixed_outdegree' - an 'outdegree' integer entry must also be in conn_spec.
                    'fixed_indegree' - an 'indegree' integer entry must also be in conn_spec.
             